Inference comparison for `TinyBert fine tuned model` with and without `Flash Attention`
===========================================

In [ ]:
# First install the dependencies
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install xformers flash-attn
!pip install -q apex

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247

In [ ]:
# Load GLUE dataset (MRPC task)
from datasets import load_dataset  # Import the load_dataset function
import evaluate


task = "mrpc"  # Set the task to "mrpc"
dataset = load_dataset("glue", task)
metric = evaluate.load("glue", task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# run this cell if you saved model files as zip
!unzip fine-tuned-model.zip -d fine-tuned-model

Archive:  fine-tuned-model.zip
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/tokenizer_config.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/tokenizer.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/special_tokens_map.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/vocab.txt  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/training_args.bin  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/trainer_state.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/adapter_config.json  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/adapter_model.safetensors  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/README.md  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/rng_state.pth  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/scheduler.pt  
  inflating: fine-tuned-model/model_files_HPML_accuracy_82/optimizer.pt  


In [ ]:

from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    "./fine-tuned-model/model-files",
    quantization_config=quantization_config,
    trust_remote_code=True
)


config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset

test_dataset = load_dataset("glue", "mrpc", split="test")

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-model/model-files", padding=True, truncation=True)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

tokenized_dataset = test_dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

test_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=32,
    collate_fn=data_collator
)

In [ ]:
# Create a custom model class that includes Flash Attention
class CustomModelWithFlashAttention(AutoModelForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        # Get hidden states from original BERT model
        hidden_states = self.bert(input_ids=input_ids, attention_mask=attention_mask, **kwargs).last_hidden_state

        for layer in self.bert.encoder.layer:
            attention_output = apply_flash_attention(layer.attention.self, hidden_states, attention_mask)
            hidden_states = layer.output.dense(attention_output)  # Pass through feed-forward and layer norm

        # Pass through the classifier head
        logits = self.classifier(hidden_states[:, 0, :])  # Use [CLS] token representation

        return logits

In [ ]:
# Instantiate the custom model
import time

custom_model = CustomModelWithFlashAttention.from_pretrained(
    "./fine-tuned-model/model-files",
    quantization_config=quantization_config,
    trust_remote_code=True
)
custom_model.to("cuda")
custom_model.eval()

latencies = []

latencies = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to GPU
        batch = {k: v.to("cuda") for k, v in batch.items()}

        # Start timing
        start_time = time.time()

        # Pass the entire batch to the custom model
        outputs = custom_model(**batch)  # This includes input_ids and attention_mask

        torch.cuda.synchronize()  # Synchronize CUDA for accurate timing
        end_time = time.time()

        latency = end_time - start_time
        latencies.append(latency)

# Calculate average latency
average_latency = sum(latencies) / len(latencies)
print(f"Average inference latency with Flash Attention: {average_latency * 1000:.2f} ms")

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average inference latency with Flash Attention: 13.15 ms


Let us run the experiment WITHOUT Flash Attention first
-------------------------------------------------------

In [ ]:
# Instantiate the original model without Flash Attention
original_model = AutoModelForSequenceClassification.from_pretrained(
    "./fine-tuned-model/model-files",
    quantization_config=quantization_config,
    trust_remote_code=True
)
original_model.to("cuda")
original_model.eval()

# Function to evaluate latency for a given model and batch size
def evaluate_latency(the_model, batch_size, test_dataset):
    # Create DataLoader for the specified batch size
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        collate_fn=data_collator
    )

    latencies = []

    with torch.no_grad():
        for batch in test_dataloader:
            # Move batch to GPU
            batch = {k: v.to("cuda") for k, v in batch.items()}
            torch.cuda.synchronize()  # Synchronize CUDA for accurate timing
            # Start timing
            start_time = time.time()

            outputs = the_model(**batch)


            torch.cuda.synchronize()  # Synchronize CUDA for accurate timing
            end_time = time.time()

            latency = end_time - start_time
            latencies.append(latency)

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies)
    return average_latency

# Test different batch sizes from 32 to 2048 (increasing by powers of 2)
batch_sizes = [32, 64, 128, 256, 512, 1024]

for batch_size in batch_sizes:
    print(f"Testing with batch size: {batch_size}")

    # Evaluate latency without Flash Attention (using original model)
    original_latency = evaluate_latency(original_model, batch_size, tokenized_dataset)
    print(f"Average inference latency without Flash Attention: {original_latency * 1000:.2f} ms")

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing with batch size: 32
Average inference latency without Flash Attention: 13.15 ms
Testing with batch size: 64
Average inference latency without Flash Attention: 13.29 ms
Testing with batch size: 128
Average inference latency without Flash Attention: 13.65 ms
Testing with batch size: 256
Average inference latency without Flash Attention: 19.40 ms
Testing with batch size: 512
Average inference latency without Flash Attention: 32.71 ms
Testing with batch size: 1024
Average inference latency without Flash Attention: 62.28 ms


Now, let us run the Experiment WITH Flash Attention
----------------------------------------------------

In [ ]:
# Instantiate the custom model with Flash Attention
custom_model = CustomModelWithFlashAttention.from_pretrained(
    "./fine-tuned-model/model-files",
    quantization_config=quantization_config,
    trust_remote_code=True
)
custom_model.config.use_cache = True
custom_model.to("cuda")
custom_model.eval()

for batch_size in batch_sizes:
    print(f"Testing with batch size: {batch_size}")

    # Evaluate latency with Flash Attention
    flash_attention_latency = evaluate_latency(custom_model, batch_size, tokenized_dataset)
    print(f"Average inference latency with Flash Attention: {flash_attention_latency * 1000:.2f} ms")


`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing with batch size: 32
Average inference latency with Flash Attention: 12.73 ms
Testing with batch size: 64
Average inference latency with Flash Attention: 12.63 ms
Testing with batch size: 128
Average inference latency with Flash Attention: 13.75 ms
Testing with batch size: 256
Average inference latency with Flash Attention: 19.25 ms
Testing with batch size: 512
Average inference latency with Flash Attention: 32.87 ms
Testing with batch size: 1024
Average inference latency with Flash Attention: 62.15 ms


In [ ]:
# Instantiate the original model without Flash Attention
base_model = AutoModelForSequenceClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_4L_312D",
)
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
base_model.to("cuda")
base_model.eval()

# Function to evaluate latency for a given model and batch size
def evaluate_latency(the_model, batch_size, test_dataset):
    # Create DataLoader for the specified batch size
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        collate_fn=data_collator
    )

    latencies = []

    with torch.no_grad():
        for batch in test_dataloader:
            # Move batch to GPU
            batch = {k: v.to("cuda") for k, v in batch.items()}
            torch.cuda.synchronize()  # Synchronize CUDA for accurate timing
            # Start timing
            start_time = time.time()

            outputs = the_model(**batch)


            torch.cuda.synchronize()  # Synchronize CUDA for accurate timing
            end_time = time.time()

            latency = end_time - start_time
            latencies.append(latency)

    # Calculate average latency
    average_latency = sum(latencies) / len(latencies)
    return average_latency

# Test different batch sizes from 32 to 2048 (increasing by powers of 2)
batch_sizes = [32, 64, 128, 256, 512, 1024]

for batch_size in batch_sizes:
    print(f"Testing with batch size: {batch_size}")

    # Evaluate latency without Flash Attention (using original model)
    base_latency = evaluate_latency(base_model, batch_size, tokenized_dataset)
    print(f"Average inference latency on Original TinyBert: {base_latency * 1000:.2f} ms")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing with batch size: 32
Average inference latency without Flash Attention on Original TinyBert: 5.01 ms
Testing with batch size: 64
Average inference latency without Flash Attention on Original TinyBert: 8.05 ms
Testing with batch size: 128
Average inference latency without Flash Attention on Original TinyBert: 14.26 ms
Testing with batch size: 256
Average inference latency without Flash Attention on Original TinyBert: 27.27 ms
Testing with batch size: 512
Average inference latency without Flash Attention on Original TinyBert: 45.59 ms
Testing with batch size: 1024
Average inference latency without Flash Attention on Original TinyBert: 84.70 ms
